In [1]:
#import libraries and the functions we defined
import os
import numpy as np
import keras
import tensorflow 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from Networks import models
from keras_unet.losses import jaccard_distance
from keras_unet.metrics import dice_coef
#from PCAUNetPP.losses import Hausdorff_distance
from sklearn.model_selection import train_test_split
import nibabel as nib
import shutil
from keras.models import load_model
from Data_Gen_2D import DataGenerator 

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [20]:
#define all metrics to evaluate model
import numpy as np
from keras import backend as K
smooth = 1

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [21]:
def gather_set(data_path, phrase):
    set_of = [f for f in os.listdir(data_path) if phrase in f]
    return np.array(set_of)


In [22]:
#filepath_predictions = r'D:\Results_CK\Compression\Numpy\PCA\Kidneys\npy'
filepath_predictions = r'D:\npy'
filepath_tensors = r'D:\t'
filepath_data = r"D:\Data_128\data\\"

images = gather_set(filepath_predictions, 'P')
model_name = 'test'

In [23]:
unique_ids = list(set([os.path.basename(image)[:14] for image in images]))
unique_ids = [name + '_' if not name.endswith('_') else name for name in unique_ids]

In [24]:
print(unique_ids)

['186714_1_93_R_', '385151_1_140_R_', '385151_1_140_L_', '186714_3_96_R_', '457036_1_105_L_', '157925_1_141_R_', '383193_3_138_L_', '183417_1_144_R_', '113994_2_99_L_', '183417_0_129_R_', '290336_3_114_L_', '295106_0_108_R_', '186714_0_78_R_', '457036_0_105_R_', '186714_3_96_L_', '385151_3_154_R_', '295106_3_120_L_', '157925_0_126_R_', '290336_2_114_R_', '283935_1_135_R_', '187456_0_87_L_', '383193_6_147_L_', '385151_0_126_L_', '385151_2_147_L_', '113994_3_108_L_', '113994_1_99_L_', '187456_3_120_L_', '295106_1_108_L_', '295106_2_120_R_', '380166_4_144_R_', '457036_2_105_R_', '295106_0_108_L_', '139486_0_126_R_', '183417_3_144_R_', '283935_2_126_R_', '383193_4_117_L_', '385151_3_154_L_', '383193_5_129_L_', '139486_3_111_R_', '290336_1_114_L_', '457036_1_105_R_', '457036_3_111_L_', '186714_2_96_L_', '457036_2_105_L_', '383193_3_138_R_', '290336_2_114_L_', '380166_3_129_R_', '295106_3_120_R_', '283935_3_114_R_', '295106_2_120_L_', '385151_0_126_R_', '383193_4_117_R_', '157925_2_144_R_', 

# Stack Images, ground truth and predicted images

In [25]:
import re
for i in range(len(unique_ids)):
    pt_info = unique_ids[i]
    pt_num, yr_num, num_slices = re.findall(r'\d+', pt_info)
    tensor = np.zeros((128,128,int(num_slices)))
    for x in range(int(num_slices)):
        img_name = unique_ids[i]+str(x)+'_C.npy'
        image = np.load(filepath_data + '\\' + img_name)
        img_slice = image
        tensor[:,:,x] = img_slice
        x = x+1
    new_fname = unique_ids[i]+'C.npy'
    np.save(os.path.join(filepath_tensors, new_fname), tensor)

In [76]:
import re
for i in range(len(unique_ids)):
    pt_info = unique_ids[i]
    pt_num, yr_num, num_slices = re.findall(r'\d+', pt_info)
    tensor = np.zeros((128,128,int(num_slices)))
    for x in range(int(num_slices)):
        img_name = unique_ids[i]+str(x)+'_M.npy'
        image = np.load(filepath_data + '\\' + img_name)
        img_slice = image
        tensor[:,:,x] = img_slice
        x = x+1
    new_fname = unique_ids[i]+'M.npy'
    np.save(os.path.join(filepath_tensors, new_fname), tensor)

In [26]:
for i in range(len(unique_ids)):
    pt_info = unique_ids[i]
    pt_num, yr_num, num_slices = re.findall(r'\d+', pt_info)
    tensor = np.zeros((128,128,int(num_slices)))
    for x in range(int(num_slices)):
        img_name = unique_ids[i]+str(x)+ '_' + model_name +'_P.npy'
        image = np.load(filepath_predictions + '\\' + img_name)
        img_slice = image[:,:,1]
        tensor[:,:,x] = img_slice
        x = x+1
    new_fname = unique_ids[i]+ model_name +'_Prediction.npy'
    np.save(os.path.join(filepath_tensors, new_fname), tensor)

Gather prediction tensors and calculate stats


In [27]:
filepath_tensors = filepath_tensors
pred_list = gather_set(filepath_tensors, '_Prediction')
true_list = gather_set(filepath_tensors, '_C.')
print(pred_list)
print(true_list)

['113994_0_87_L_test_Prediction.npy' '113994_1_99_L_test_Prediction.npy'
 '113994_2_99_L_test_Prediction.npy' '113994_3_108_L_test_Prediction.npy'
 '139486_0_126_R_test_Prediction.npy' '139486_1_99_R_test_Prediction.npy'
 '139486_2_99_R_test_Prediction.npy' '139486_3_111_R_test_Prediction.npy'
 '157925_0_126_R_test_Prediction.npy' '157925_1_141_R_test_Prediction.npy'
 '157925_2_144_R_test_Prediction.npy' '157925_3_144_R_test_Prediction.npy'
 '183417_0_129_R_test_Prediction.npy' '183417_1_144_R_test_Prediction.npy'
 '183417_2_144_R_test_Prediction.npy' '183417_3_144_R_test_Prediction.npy'
 '186714_0_78_L_test_Prediction.npy' '186714_0_78_R_test_Prediction.npy'
 '186714_1_93_L_test_Prediction.npy' '186714_1_93_R_test_Prediction.npy'
 '186714_2_96_L_test_Prediction.npy' '186714_2_96_R_test_Prediction.npy'
 '186714_3_96_L_test_Prediction.npy' '186714_3_96_R_test_Prediction.npy'
 '187456_0_87_L_test_Prediction.npy' '187456_1_120_L_test_Prediction.npy'
 '187456_2_120_L_test_Prediction.npy' '

In [28]:
name =pred_list[0]
print(name[18:46])
test = pred_list[0][:17]+'C.npy'
print(test)

_Prediction.npy
113994_0_87_L_tesC.npy


In [29]:
results = []
results1 = []
for i in range(len(pred_list)):
    prediction = np.load(filepath_tensors + '\\'+ pred_list[i])
    true = np.load(filepath_tensors + '\\'+true_list[i])
    dice_calc = dice_coef(true,prediction)
    #model = pred_list[i][:-6]
    patient = pred_list[i][:-6]
    new_calc = [dice_calc.numpy()]
    new_calc1 = [patient, dice_calc.numpy()]
    results.append(new_calc)
    results1.append(new_calc1)
  
    
    

In [30]:
import math
min_values = []
max_values = []
for col in zip(*results):
    col_values = [float(val()) if callable(val) else float(val) for val in col]
    col_min = min(col_values)
    col_max = max(col_values)
    min_values.append(col_min)
    max_values.append(col_max)

# patient_min = min_values[0]  # Minimum value of patient values
dice_calc_min = min_values[0]  # Minimum value of dice_calc values
dice_calc_max = max_values[0]
print("Minimum Dice Coefficient:", dice_calc_min)
print("Minimum Dice Coefficient:", dice_calc_max)
averages = []
for col in zip(*results):
    col_values = [float(val()) if callable(val) else float(val) for val in col]
    col_avg = sum(col_values) / len(col_values)
    averages.append(col_avg)
dice_calc_avg = averages[0]
# Calculate standard deviation for each parameter
dice_calc_sd = math.sqrt(sum((float(val()) if callable(val) else float(val) - dice_calc_avg) ** 2 for val in results[0]) / len(results[0]))

print("Dice Coefficient Average:", dice_calc_avg)
print("Dice Coefficient Standard Deviation:", dice_calc_sd)

Minimum Dice Coefficient: 0.7170935117125993
Minimum Dice Coefficient: 0.928668623125065
Dice Coefficient Average: 0.8686934824283568
Dice Coefficient Standard Deviation: 0.014869776147047653


In [67]:
print(results)

[['113994_0_87_L_tes', 0.8835635305066377], ['113994_1_99_L_tes', 0.8914922152092589], ['113994_2_99_L_tes', 0.8564934053918366], ['113994_3_108_L_te', 0.8934202600974314], ['139486_0_126_R_te', 0.9222143150392268], ['139486_1_99_R_tes', 0.9082955230969956], ['139486_2_99_R_tes', 0.9286691555879607], ['139486_3_111_R_te', 0.9129185161860741], ['157925_0_126_R_te', 0.8985464396452444], ['157925_1_141_R_te', 0.8952782078004687], ['157925_2_144_R_te', 0.883488930106251], ['157925_3_144_R_te', 0.9049351171785605], ['183417_0_129_R_te', 0.8453222321389027], ['183417_1_144_R_te', 0.8333218547638429], ['183417_2_144_R_te', 0.8695757761438256], ['183417_3_144_R_te', 0.8387048906656192], ['186714_0_78_L_tes', 0.889412605976103], ['186714_0_78_R_tes', 0.8963161398192137], ['186714_1_93_L_tes', 0.8939967145485421], ['186714_1_93_R_tes', 0.8807512055201604], ['186714_2_96_L_tes', 0.8964511230274046], ['186714_2_96_R_tes', 0.8918503689453008], ['186714_3_96_L_tes', 0.9014284297581412], ['186714_3_9

In [53]:
import pandas as pd
df = pd.DataFrame(results1)
df.columns =[ 'Patient','Dice']
filepath = r"D:\Results_CK\sUNet++\Cyst\performance.xlsx"
df.to_excel(filepath, index=False)

# Convert predictions to nii for visualization

In [56]:
def gather_images(data_path):
    images = []
    path = data_path + '\\'
    for f in os.listdir(data_path):
      if '_test' in f:
        images.append(f)
        
      else:
        continue
    images = np.array(images)
    #segmentations = np.array(segmentations)

    indices = np.array(range(len(images))) # we will use this in the next step.

    return images

In [57]:
filepath_tensors = r'D:\t'
images = gather_images(filepath_tensors)
print(len(images))
path = r'D:\Results_CK\sUNet++\Cyst\nii'
for i in range(len(images)):
    image = np.load(os.path.join(filepath_tensors, images[i]))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(image, affine)
    filename = images[i][:-3]+'nii'
    nib.save(nifti_file, os.path.join(path, filename))

76
